<a href="https://colab.research.google.com/github/piarosebelledelapaz/DUGAN/blob/main/pdf_anonymization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 48.0 MB/s eta 0:00:00


In [ ]:
#import PyMuPDF

import fitz
from io import BytesIO
import os

In [ ]:
# a function that searches for a specified keyword (a string) in every line of the pdf

def search_for_keyword(page_lines, search_str):
    matched_values = [] # stores the line number (starting from 1) and the keyword as a tuple
    for line_number, line in enumerate(page_lines):
        if search_str in line:
            matched_values.append((line_number + 1, line.strip()))
    print(matched_values)
    return matched_values

In [ ]:
# a function that anonymizes the pdf by redacting the nearby field of the matched keyword found on the page with a black rectangle

def anonymize(page, matched_values, page_number):
    matches_found = 0
    for line_number, line_text in matched_values:
        matches_found += 1

        keyword_coor = page.search_for(line_text)[0] # the exact position of the word in the pdf, (x0, y0, x1, y1)

        # variables to adjust the coordinates
        line_width = keyword_coor.width + 50
        line_height = keyword_coor.y1 - keyword_coor.y0 + 5
        shift_amount = 5

        new_rect = fitz.Rect(keyword_coor.x1, keyword_coor.y0 - shift_amount, keyword_coor.x1+line_width, keyword_coor.y0+line_height- shift_amount) #redacted area

        page.draw_rect(new_rect, color=(0, 0, 0), fill=(0, 0, 0))

        # print(f"Keyword_coor: {keyword_coor}\nKeyword: {line_text}\nPage: {page_number}\nCoordinates: {new_rect}\n")

    return matches_found

In [ ]:
# function to process the pdf file to be anonymized

def process_pdf(input_file: str, search_strings: list):
    pdfDoc = fitz.open(input_file)

    output_buffer = BytesIO()
    total_matches = 0

    for pg in range(pdfDoc.page_count):
        page = pdfDoc[pg]
        page_lines = page.get_text("text").split('\n')

        for search_str in search_strings:
            matched_values = search_for_keyword(page_lines, search_str)
            if matched_values:
                matches_found = anonymize(page, matched_values, pg + 1)
                total_matches += matches_found

    input_filename, input_file_extension = os.path.splitext(os.path.basename(input_file))
    output_file = f"{input_filename}_anonymized.pdf"

    pdfDoc.save(output_buffer)
    pdfDoc.close()

    with open(output_file, mode='wb') as f:
        f.write(output_buffer.getbuffer())

In [ ]:
process_data("/content/test.pdf", ["Name", "Vorname", "Geburtsdatum"])

Keyword_coor: Rect(11.338600158691406, 590.7291870117188, 36.23460006713867, 600.2332153320312)
Keyword: Name:
Page: 1
Coordinates: Rect(36.23460006713867, 585.7291870117188, 111.13059997558594, 600.2332153320312)

Keyword_coor: Rect(201.33859252929688, 590.7291870117188, 237.59458923339844, 600.2332153320312)
Keyword: Vorname:
Page: 1
Coordinates: Rect(237.59458923339844, 585.7291870117188, 323.8505859375, 600.2332153320312)

Keyword_coor: Rect(11.338600158691406, 615.7291870117188, 68.84260559082031, 625.2332153320312)
Keyword: Geburtsdatum:
Page: 1
Coordinates: Rect(68.84260559082031, 610.7291870117188, 176.34661102294922, 625.2332153320312)

Keyword_coor: Rect(373.6852722167969, 783.2598266601562, 393.9132995605469, 790.9818115234375)
Keyword: Name:
Page: 2
Coordinates: Rect(393.9132995605469, 778.2598266601562, 464.1413269042969, 790.9818115234375)

Keyword_coor: Rect(228.455810546875, 783.2598266601562, 257.913818359375, 790.9818115234375)
Keyword: Vorname:
Page: 2
Coordinates: R